In [ ]:
import sys
input=sys.stdin.readline

def sol() :
  C, N = map(int, input().split())
  colors: list[str] = list(input().rstrip() for _ in range(C))
  nick: list[str] = list(input().rstrip() for _ in range(N))
  colors.sort() #사전순으로 정렬해서 먼저 찾을 수 있는 것을 찾는다.
  nick.sort()

  Q = int(input())
  ans = []
  for _ in range(Q) :
    team = input().rstrip()
    for c in colors:
      x = team.find(c)
      if x == -1 : continue
      ans.append("Yes" if any(n in team[x+len(c):] for n in nick) else "No")
      break
    else :
      ans.append("No")
  
  sys.stdout.write('\n'.join(ans))

sol()

4 3
red
blue
purple
orange
shift
joker
noon
5
redshift
bluejoker
purplenoon
orangeshift
whiteblue
;;yes * 4
2 2
a
aa
a
b
4
aa
ab
aab
bab

### 아이디어
- Python의 string.find()가 $\Omicron(N+M)$ 이므로 KMP를 구현할 필요를 느끼지 못하겠다.
- 나이브하게 $\Omicron(CNQ)$ 로 구하는 코드를 구현했다. 물론 TLE가 날 것이다.
- 생각한 반례중, 팀명이나 색상명이 어떤 문자열의 부분 문자열로 존재하는 경우에 대해서 잘못 처리할 수도 있을 것이라는 생각이 들었다.
- 두번째 반례는 색상 이후에 이름이 나와야 하는데, 색상이 끝나지 않은 상태에서 이름이 나오는 경우이다.

In [ ]:
import sys
input=sys.stdin.readline

def sol() :
  C, N = map(int, input().split())
  colors: list[str] = list(input().rstrip() for _ in range(C))
  nick: list[str] = list(input().rstrip() for _ in range(N))
  colors.sort() #사전순으로 정렬해서 먼저 찾을 수 있는 것을 찾는다.
  nick.sort()

  Q = int(input())
  res = [float("inf")] * Q #닉네임을 찾기 시작할 가능한 가장 왼쪽의 인덱스
  ans = []
  for i in range(Q) :
    team = input().rstrip()
    for c in colors:
      x = team.find(c)
      if x == -1 : continue
      res[i] = min(res[i], x+len(c))

    if res[i] == float("inf") : 
      ans.append("No")
      break #색깔을 못찾았으면 무조건 실패
    
    for n in nick :
      x = team.find(n, res[i])
      if x == -1 : continue
      ans.append("Yes")
      break
    else :
      ans.append("No")
  
  sys.stdout.write('\n'.join(ans))

sol()

- $\Omicron(Q(C+N))$ 으로 단축했지만 시간초과를 면하지 못했다. 
- 애초에 문자열 길이가 엄청나게 긴데 왜 KMP를 생각했는지 모르겠다. 분명 트라이문제 일 것이다.
  - 실제 시간복잡도는, 닉네임과 색상의 최대 길이 m, 팀명의 최대길이 r이라 할때 $\Omicron(Q(2m+r)(C+N))$ 이다..

In [ ]:
import sys
input=sys.stdin.readline

class Trie:
  def __init__(self, l):
    self.r = {}
    for s in l:
      self.add(s)

  def add(self, s):
    cur = self.r
    for c in s:
      cur = cur.setdefault(c, {})
    cur["_end_"] = True

  def find(self, s) : #return s index
    cur = self.r
    for i, c in enumerate(s):
      if c not in cur:         continue
      cur = cur[c]
      if "_end_" in cur:
        return i
    else :
      return -1

def sol() :
  C, N = map(int, input().split())
  colors: list[str] = list(input().rstrip() for _ in range(C))
  nick: list[str] = list(input().rstrip() for _ in range(N))
  CT =  Trie(colors)
  NT = Trie(nick)

  Q = int(input())
  res = [-1] * Q #닉네임을 찾기 시작할 가능한 가장 왼쪽의 인덱스
  ans = []
  for i in range(Q) :
    team = [*input().rstrip()]
    x = CT.find("".join(team))
    res[i] = x

    if res[i] == -1 or res[i] == len(team): 
      ans.append("No")
      break #색깔을 못찾았으면 무조건 실패
    ans.append("Yes" if NT.find("".join(team[res[i]+1:])) != -1 else "No")
  
  sys.stdout.write('\n'.join(ans))

sol()

- 파이썬으로 간신히 뚫리는 것 같은데 WA가 났다. 아무래도 가능한 가장 짧은 문자열을 찾는 부분이 잘못된 것 같다.
  - 트라이상에서 먼저 찾아지는 문자열의 길이가 매우 긴 반례를 생각할 수 있다. 더 뒤에 있지만 더 짧은 문자열이 존재하고, 그것을 골라야할때 문제가 생길 것이다.
  - 그렇다고 트라이 상에서 bfs로 찾으려다간 TLE가 또 날것 같은데 어떻게 해야할까?

4 3
red
blue
purple
orange
shift
joker
noon
5
redshift
bluejoker
purplenoon
orangeshift
whiteblue
;;Yes * 4
2 2
a
aa
a
b
4
aa
ab
aab
bab
;; No
1 1
dabc
cabc
1
dabcabc
;; Yes
1 1
xb
xa
1
xbxac
;; Yes
2 1
abcdefg
cd
e
1
abcde

In [ ]:
import sys, io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class Trie:
  def __init__(self, l):
    self.r = {}
    for s in l:
      self.add(s)

  def add(self, s):
    cur = self.r
    for c in s:
      cur = cur.setdefault(c, {})
    cur["_end_"] = True

  def find(self, s, res) :
    cur = self.r
    for i, c in enumerate(s):
      if c not in cur: return
      cur = cur[c]
      if "_end_" in cur:
        res[i] = True

def sol() :
  C, N = map(int, input().split())
  colors: list[str] = list(input().rstrip() for _ in range(C))
  nick: list[str] = list(input().rstrip()[::-1] for _ in range(N))
  CT =  Trie(colors)
  NT = Trie(nick)

  Q = int(input())
  ans = []
  for _ in range(Q) :
    team = input().rstrip()
    len_team = len(team)
    cres = [False] * len_team
    rres = [False] * len_team
    CT.find(team, cres)
    NT.find(team[::-1], rres)
    rres.reverse()
    ans.append("Yes" if any(cres[i] and rres[i+1] for i in range(len_team-1)) else "No")
  
  sys.stdout.write('\n'.join(ans))

sol()

- 문제 이해를 제대로 못한걸지도 모른다.. 왜 맞는지 모르겠다.
  - (쓰레기문자)(색깔)(닉네임) 순으로 적혀있으면 하나도 거르지 못하는 것이 아닌가? 문제에는 쓰레기문제가 없을 것이라는 단서를 찾을 수 없다.
  - 색상 이름과 닉네임의 순서로 `이어서`가 `두 문자열이 연속하게` 있어야 한다는 내용이라는 점을 확실하게 캐치하지 못했다. 어휘가 딸린거였을까?